In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
nac = pd.read_csv('../Data/nac2018.csv',sep=';',dtype='object',nrows=10000)
fet = pd.read_csv('../Data/fetal2018.csv',sep=';',dtype='object',nrows=10000)
nofet = pd.read_csv('../Data/nofetal2018.csv',sep=';',dtype='object')#,nrows=40000) 

## Rename some columns: Change (AREANAC, A_DEFUN,...) -> AREA

Convencion: todas variables se refieren al parto (lugar, mes, etc) a menos que se especifique

In [3]:
nac.columns = ['COD_DPTO', 'COD_MUNIC', 'AREA', 'SIT', 'OTRO_SIT', 'SEXO',
       'PESO', 'TALLA', 'ANO', 'MES', 'PARTO_ATENDIDO_POR', 'TIEMPO_GESTACION', 'NUM_CONSUL',
       'TIPO_PARTO', 'TIPO_EMBARAZO', 'APGAR1', 'APGAR2', 'IDHEMOCLAS',
       'IDFACTORRH', 'IDPERTET', 'EDAD_MADRE', 'EST_CIVM', 'NIV_EDUM',
       'ULTCURMAD', 'PAIS_RESIDENCIA', 'DEPTO_RESIDENCIA', 'MUN_RECIDENCIA', 'AREA_RESIDENCIA_HAB', 'N_HIJOSV',
       'FECHA_NACM', 'N_EMB', 'SEG_SOCIAL', 'IDCLASADMI', 'EDAD_PADRE',
       'NIV_EDUP', 'ULTCURPAD', 'PROFESION']

fet.columns = ['COD_DPTO', 'COD_MUNIC', 'AREA', 'SIT', 'OTRO_SIT',
       'RESULTADO_EMB', 'ANO', 'MES', 'HORA', 'MINUTOS', 'SEXO', 'PAIS_RESIDENCIA',
       'DEPTO_RESIDENCIA', 'MUN_RECIDENCIA', 'AREA_RESIDENCIA_HAB', 'SEG_SOCIAL', 'IDADMISALUD',
       'MANERA_MUERTE', 'CERT_EXPEDIDO_POR', 'MOMENTO_MUERTE', 'TIPO_PARTO', 'TIPO_EMBARAZO', 'TIEMPO_GESTACION',
       'PESO', 'EDAD_MADRE', 'N_HIJOSV', 'N_HIJOSM', 'EST_CIVM',
       'NIV_EDUM', 'ULTCURMAD', 'MANERA_MUERTE_VIOLENTA', 'CODOCUR', 'CODMUNOC', 'C_MUERTEA',
       'C_MUERTEB', 'C_MUERTEC', 'C_MUERTED', 'C_MUERTEE', 'ASIS_MED',
       'C_DIR1', 'C_DIR12', 'C_ANT1', 'C_ANT12', 'C_ANT2', 'C_ANT22', 'C_ANT3',
       'C_ANT32', 'C_PAT1', 'C_PAT2', 'C_BAS1', 'C_MCM1', 'CAUSA_667',
       'PROFESION', 'CAU_HOMOL']

nofet.columns = ['COD_DPTO', 'COD_MUNIC', 'AREA', 'SIT', 'OTRO_SIT',
       'RESULTADO_EMB', 'ANO', 'MES', 'HORA', 'MINUTOS', 'SEXO', 'EST_CIVIL',
       'GRU_ED1', 'GRU_ED2', 'NIVEL_EDU_FALLECIDO', 'ULTCURFAL', 'MUERTEPORO',
       'SIMUERTEPO', 'OCUPACION', 'IDPERTET', 'PAIS_RESIDENCIA', 'DEPTO_RESIDENCIA',
       'MUN_RECIDENCIA', 'AREA_RESIDENCIA_HAB', 'SEG_SOCIAL', 'IDADMISALUD', 'MANERA_MUERTE',
       'CERT_EXPEDIDO_POR', 'MOMENTO_MUERTE', 'TIPO_PARTO', 'TIPO_EMBARAZO', 'TIEMPO_GESTACION', 'PESO',
       'EDAD_MADRE', 'N_HIJOSV', 'N_HIJOSM', 'EST_CIVM', 'NIV_EDUM',
       'ULTCURMAD', 'EMB_FAL', 'EMB_SEM', 'EMB_MES', 'MANERA_MUERTE_VIOLENTA', 'CODOCUR',
       'CODMUNOC', 'C_MUERTEA', 'C_MUERTEB', 'C_MUERTEC', 'C_MUERTED',
       'C_MUERTEE', 'ASIS_MED', 'C_DIR1', 'C_DIR12', 'C_ANT1', 'C_ANT12',
       'C_ANT2', 'C_ANT22', 'C_ANT3', 'C_ANT32', 'C_PAT1', 'C_PAT2', 'C_BAS1',
       'C_MCM1', 'CAUSA_667', 'PROFESION', 'CAU_HOMOL']

nac['RESULTADO_EMB'] = 3
nac["NacidoVivo"] = 1
fet["NacidoVivo"] = 0
nofet["NacidoVivo"] = 0

Otro sitio: No nació en hospital, donde?

IDPERTET: Raza, cultura.

PROFESION: Prof. de quien certifica nacimiento

IDADMISALUD ??

FECHA_NACM: Fecha de nacim ultimo hijo vivo

APGAR: APGAR test values viablility of newborn https://es.wikipedia.org/wiki/Test_de_Apgar

IDHEMOCLAS + IDFACTORRH: Blood type

nofetal df contiene datos de madre. murio? como? etc. (EMB_FAL,EMB_SEM ?)

Estado civil del muerto ???

GRU_EDU{1,2}: edad del fallecido. mayor detalle en 1

all other C_{} are death causes

In [4]:
nac.head()

,COD_DPTO,COD_MUNIC,AREA,SIT,OTRO_SIT,SEXO,PESO,TALLA,ANO,MES,...,N_HIJOSV,FECHA_NACM,N_EMB,SEG_SOCIAL,IDCLASADMI,EDAD_PADRE,NIV_EDUP,ULTCURPAD,PROFESION,TIPO_DEFUN
0,05,001,1,1,,1,7,5,2018,01,...,2,13/08/2014,2,1,1,30,04,11,1,0
1,11,001,1,1,,1,6,5,2018,01,...,1,,1,1,1,36,09,5,1,0
2,52,001,1,1,,1,6,5,2018,01,...,2,15/02/2011,3,2,2,27,04,11,1,0
3,11,001,1,1,,1,6,5,2018,01,...,1,,2,1,1,32,09,5,1,0
4,50,001,1,1,,1,5,4,2018,01,...,1,,1,2,2,21,04,11,1,0


In [5]:
fet.head()

,COD_DPTO,COD_MUNIC,AREA,SIT,OTRO_SIT,TIPO_DEFUN,ANO,MES,HORA,MINUTOS,...,C_ANT22,C_ANT3,C_ANT32,C_PAT1,C_PAT2,C_BAS1,C_MCM1,CAUSA_667,PROFESION,CAU_HOMOL
0,86,568,1,1,,1,2018,01,21,03,...,,,,,,P018,,402,1,080
1,41,298,1,1,,1,2018,01,00,00,...,,,,,,P018,,402,1,080
2,11,001,1,1,,1,2018,01,00,00,...,,,,,,P018,,402,1,080
3,52,399,1,9,,1,2018,01,00,00,...,,,,P018,,P018,,402,1,080
4,41,298,1,1,,1,2018,01,00,00,...,,,,,,P018,,402,1,080


In [6]:
nofet.head()

,COD_DPTO,COD_MUNIC,AREA,SIT,OTRO_SIT,TIPO_DEFUN,ANO,MES,HORA,MINUTOS,...,C_ANT22,C_ANT3,C_ANT32,C_PAT1,C_PAT2,C_BAS1,C_MCM1,CAUSA_667,PROFESION,CAU_HOMOL
0,17,050,1,5,,2,2018,01,15,05,...,,,,,,I219,,303,1,051
1,18,592,1,1,,2,2018,01,08,15,...,,,,R011,,E43,,602,1,042
2,17,013,1,3,,2,2018,01,10,00,...,,,,I10,E785,I219,,303,1,051
3,05,001,1,3,,2,2018,01,12,15,...,,,,,,C20,,203,1,014
4,50,001,1,1,,2,2018,01,11,00,...,K388,,,I119,,K352,,607,1,065


### El df nofet contiene datos de muertes de mayores de edad. A partir de aquí: seleccionar solo las entradas correspondientes a menores de 1 año. (GRU_ED1 < 07 , GRU_ED2 = 01)

In [7]:
#Asegurarse que variables son consistentes (deberían tener igual cantidad de entradas para este query)
print(nofet[nofet['GRU_ED2'] == '01'].shape)
print(nofet[nofet['GRU_ED1'].astype(int) < 7].shape)

#That checked, slice the df
nofet = nofet[nofet['GRU_ED2'] == '01']
nofet.head()

(7329, 66)
(7329, 66)


,COD_DPTO,COD_MUNIC,AREA,SIT,OTRO_SIT,TIPO_DEFUN,ANO,MES,HORA,MINUTOS,...,C_ANT22,C_ANT3,C_ANT32,C_PAT1,C_PAT2,C_BAS1,C_MCM1,CAUSA_667,PROFESION,CAU_HOMOL
11,44,430,1,1,,2,2018,01,09,40,...,,E872,,,,E46,,602,1,042
183,81,736,1,1,,2,2018,01,04,57,...,,,,A879,E46,A879,,105,1,010
185,17,001,1,1,,2,2018,01,09,00,...,,Q349,,Q249,,Q349,,613,1,088
229,73,001,1,1,,2,2018,01,03,30,...,,J398,Q320,Q909,,Q320,,613,1,088
418,08,001,1,1,,2,2018,02,19,20,...,,,,P271,,P77,,406,1,085




## Replacement dicts


In [ ]:
replacements = {
    
    "TIPO_PARTO": {
        1 : "ESPONTANEO",
        2 : "CESAREA",
        3 : "INSTRUMENTADO",
        4 : "IGNORADO",
        9 : "SIN_INFORMACION"
    },
    
    "SEG_SOCIAL": {
        1 : "CONTRIBUTIVO",
        2 : "SUBSIDIADO",
        3 : "EXCEPCION",
        4 : "ESPECIAL",
        5 : "NO_ASEGURADO",
        9 : "SIN_INFORMACION"
    },
    
    "MANERA_MUERTE": {
        0: "SIN_INFORMACION",
        1 : "NATURAL",
        2 : "VIOLENTA",
        3 : "EN_ESTUDIO"
    },
    
    "SITIO_EVENTO":{
        1 : 'INSTITUCION DE SALUD',
        3 : 'CASA',
        4 : 'TRABAJO',
        5 : 'VIA PUBLICA',
        6 : 'OTRO',
        9 : 'SIN_INFORMACION'
    },    
    
    "CERT_EXPEDIDO_POR":{
        1 : "MEDICO_TRATANTE",
        2 : "MEDICO_NO_TRATANTE",
        3 : "MEDICO_LEGISTA",
        4 : "PERSONAL_AUTORIZADO",
        5 : "FUNCIONARIO_REG_CIVIL",
        6 : "OTRO"
    },
    
    "MOMENTO_MUERTE" : {
        1 : "ANTES",
        2 : "DURANTE",
        3 : "DESPUES",
        4 : "IGNORADO",
        9 : "SIN_INFORMACION"
    },
    
    "TIPO_EMBARAZO" : {
        1 : "SIMPLE",
        2 : "DOBLE",
        3 : "TRIPLE",
        4 : "CUADRUPLE_MAs",
        5 : "IGNORADO"
    },
    
    "TIEMPO_GESTACION" : {
        1 : "0-22",
        2 : "22-27",
        3 : "28-37",
        4 : "38-41",
        5 : "42-MAS",
        6 : "IGNORADO",
        9 : "SIN_INFORMACION"
    },
    
    "MANERA_MUERTE_VIOLENTA":{
        0 : "NO_VIOLENTA",
        1 : "SUICIDIO",
        2 : "HOMICIDIO",
        3 : "ACCIDENTE_TRANSITO",
        4 : "OTRO_ACCIDENTE",
        5 : "EN_ESTUDIO"
    },
    
    "PARTO_ATENDIDO_POR" : {
        1 : "MEDICO",
        2 : "ENFERMERA",
        3 : "AUXILIAR_ENFERMERIA",
        4 : "PROMOTOR_SALUD",
        5 : "PARTERA",
        6 : "OTRO",
        9 : "SIN_INFORMACION"
    },
    
    "RESULTADO_EMB" : {
        1 : "DEFUNCION_FETAL",
        2 : "DEFUNCION_NO_FETAL",
        3 : "NACIDO_VIVO"
    },
    
    "SEXO": {
        1 : "MASCULINO",
        2 : "FEMENINO",
        3 : "INDETERMINADO",
    }
}

# Estandarizar sitio de parto (defunción)

### fet y nofet:
Sitio donde ocurrió la defunción
1 Hospital/clínica
2 Centro /puesto de salud
3 Casa /domicilio
4 Lugar de trabajo
5 Vía pública
6 Otro
9 Sin información

### nac:
Sitio de la Parto
1 = Institución de salud
2 = Domicilio
3 = Otro
9 = Sin información

In [8]:
#Mezcla hospital y centro de salud en fet y nofet
fet['SIT'].replace('2','1',inplace=True)
nofet['SIT'].replace('2','1',inplace=True)

nac['SIT'].replace('3','6',inplace=True) #otro = 6
nac['SIT'].replace('2','3',inplace=True) #domicilio = 3

dfs = [nac,fet,nofet]

#dict para reemplazar por nombres

#SITIO EVENTO: nueva columna para poner todos los valores (sumariza SIT y OTRO_SIT)
for df_i in dfs:
    df_i['OTRO_SIT'].replace(' ','SIN_INFORMACION')
    df_i['SITIO_EVENTO'] = df_i['SIT'].astype('object')
    #Reemplaza valores en "replacing"
    df_i['SITIO_EVENTO'].replace(replacements["SITIO_EVENTO"].keys(),replacements["SITIO_EVENTO"].values(),inplace=True)
    #Simple cleaning
    df_i['OTRO_SIT'].replace(' ','SIN_INFORMACION',inplace=True) 
    df_i['OTRO_SIT'] = df_i['OTRO_SIT'].str.upper().str.strip()

### Concatenate all 3 dfs. From now on I assume all data is standarized among dfs

In [9]:
df = (pd.concat([nac,fet,nofet],sort=True)  
      .replace(' ',np.nan)    #replace missing values
      .reset_index(drop=True)) #reset index and drop old indices
df.shape

(27329, 80)

In [10]:
df.columns

Index(['ANO', 'APGAR1', 'APGAR2', 'AREA', 'AREA_RES', 'ASIS_MED', 'ATEN_PAR',
       'CAUSA_667', 'CAU_HOMOL', 'CODMUNOC', 'CODMUNRE', 'CODOCUR', 'CODPRES',
       'CODPTORE', 'COD_DPTO', 'COD_MUNIC', 'CONS_EXP', 'C_ANT1', 'C_ANT12',
       'C_ANT2', 'C_ANT22', 'C_ANT3', 'C_ANT32', 'C_BAS1', 'C_DIR1', 'C_DIR12',
       'C_MCM1', 'C_MUERTEA', 'C_MUERTEB', 'C_MUERTEC', 'C_MUERTED',
       'C_MUERTEE', 'C_PAT1', 'C_PAT2', 'EDAD_MADRE', 'EDAD_PADRE', 'EMB_FAL',
       'EMB_MES', 'EMB_SEM', 'EST_CIVIL', 'EST_CIVM', 'FECHA_NACM', 'GRU_ED1',
       'GRU_ED2', 'HORA', 'IDADMISALUD', 'IDCLASADMI', 'IDFACTORRH',
       'IDHEMOCLAS', 'IDPERTET', 'MAN_MUER', 'MES', 'MINUTOS', 'MUERTEPORO',
       'MUL_PARTO', 'MU_PARTO', 'NIVEL_EDU_FALLECIDO', 'NIV_EDUM', 'NIV_EDUP',
       'NUMCONSUL', 'N_EMB', 'N_HIJOSM', 'N_HIJOSV', 'OCUPACION', 'OTRO_SIT',
       'PESO', 'PMAN_MUER', 'PROFESION', 'SEG_SOCIAL', 'SEXO', 'SIMUERTEPO',
       'SIT', 'SITIO_EVENTO', 'TALLA', 'TIPO_DEFUN', 'TIPO_PARTO', 'T_GES',
   

## Fill N_EMB, N_HIJOSV and N_HIJOSM. Use N_EMB = N_HIJOSV + N_HIJOSM

In [11]:
#Convert these columns to float
df[['N_EMB','N_HIJOSV','N_HIJOSM']] = df[['N_EMB','N_HIJOSV','N_HIJOSM']].astype(float)

#Print number of missing values in these columns
print('Before:\n',df[['N_EMB','N_HIJOSV','N_HIJOSM']].isna().sum())  

df_tmp = df[df['N_EMB'].isna()]  #df with nan values for N_EMB
df.loc[df['N_EMB'].isna(),['N_EMB']] = df_tmp['N_HIJOSM'] + df_tmp['N_HIJOSV']

df_tmp = df[df['N_HIJOSM'].isna()]  #df with nan values for N_HIJOSM
df.loc[df['N_HIJOSM'].isna(),['N_HIJOSM']] = df_tmp['N_EMB'] - df_tmp['N_HIJOSV']

df_tmp = df[df['N_HIJOSV'].isna()]  #same for N_HIJOSV
df.loc[df['N_HIJOSV'].isna(),['N_HIJOSV']] = df_tmp['N_EMB'] - df_tmp['N_HIJOSM']

df.replace('NO_INFONO_INFO','NO_INFO',inplace=True)
print('\nAfter:\n',df[['N_EMB','N_HIJOSV','N_HIJOSM']].isna().sum()) 

Before:
 N_EMB       17329
N_HIJOSV        0
N_HIJOSM    10000
dtype: int64

After:
 N_EMB       0
N_HIJOSV    0
N_HIJOSM    0
dtype: int64


## Create unique code for munic.

### dpto: 05 + munic: 001 => 05001 new code

In [12]:
#Create unique munic and dpto codes (nacim, resid, death)
def id_municip(df,coldpto,colmunic,nameNew):
    #Convert dpt. and munic. codes to strings and fill with zeros   
    df[coldpto] = df[coldpto].apply(lambda x : str(x).zfill(2))
    df[colmunic] = df[colmunic].apply(lambda x : str(x).zfill(3))
    #Create a single ID for each munic.
    df[nameNew] = df[coldpto] + df[colmunic]
    return df.drop(colmunic,axis=1)  #Drop munic. code, the new code is enough

df = id_municip(df,'COD_DPTO','COD_MUNIC','ID_BIRTH')
df = id_municip(df,'DEPTO_RESIDENCIA', 'MUN_RECIDENCIA','ID_RESID')
df = id_municip(df,'CODOCUR','CODMUNOC','ID_VIOL').replace('nannan',np.nan)

df.shape

(27329, 80)

In [13]:
cols = [c for c in df.columns if c.startswith("C_M")]
df[(~df[cols].isnull()).any(axis=1)][cols].head()

,C_MCM1,C_MUERTEA,C_MUERTEB,C_MUERTEC,C_MUERTED,C_MUERTEE
10000,NaN,NaN,1,NaN,NaN,NaN
10001,NaN,NaN,1,NaN,NaN,NaN
10002,NaN,NaN,1,NaN,NaN,NaN
10003,NaN,NaN,1,NaN,NaN,NaN
10004,NaN,NaN,1,NaN,NaN,NaN


In [14]:
#Unir C_MUERTE(A,B..,E) en una sola columna. Esta variable es como se determinó la causa de muerte
#Tal vez esto no es tan importante, así que las voy a unir y tirar las otras
cols = [f'C_MUERTE{i}' for i in ['A','B','C','D','E']]
df['C_MUERTE'] = df[cols].fillna('_').astype(str).sum(axis=1)
df.drop(cols,axis=1,inplace=True)
df.shape
df["C_MUERTE"].unique()

array(['_____', '_1___', '_11__', '___1_', '__1__', '_1_1_', '11___',
       '1____', '__11_', '_111_', '11_1_', '111__', '1111_', '1_1__',
       '1__1_', '____1'], dtype=object)

In [15]:
#Crear una sola columna para DATE
print(df[['ANO', 'MES', 'HORA', 'MINUTOS']].isna().any())  #Make sure only HORA, MINUTOS are missing

#Convertir data a strings
tmp = df[['ANO', 'MES', 'HORA', 'MINUTOS']].fillna(0).astype(int).astype(str)
#Sumar con formato Y/M/D H:Min
date = tmp['ANO'] + '/' + tmp['MES'] + ' ' + tmp['HORA'] + ':' + tmp['MINUTOS']   

df['DATE'] = pd.to_datetime(date) #Convert to datetime
df.drop(['ANO', 'MES', 'HORA', 'MINUTOS'],axis=1,inplace=True)  #Drop old columns
df["DATE"].head()

ANO        False
MES        False
HORA        True
MINUTOS     True
dtype: bool


0   2018-01-01
1   2018-01-01
2   2018-01-01
3   2018-01-01
4   2018-01-01
Name: DATE, dtype: datetime64[ns]

In [16]:
## For now, fill missing values

#for this vars: no data == 9
vars9 = ['PESO','TALLA','PARTO_ATENDIDO_POR','TIEMPO_GESTACION','TIPO_PARTO','TIPO_EMBARAZO','IDHEMOCLAS', 
         'IDFACTORRH','IDPERTET','EST_CIVM','AREA_RESIDENCIA_HAB','IDCLASADMI','ULTCURPAD',
         'CERT_EXPEDIDO_POR','MOMENTO_MUERTE','MANERA_MUERTE_VIOLENTA','ASIS_MED','EST_CIVIL',]
for var in vars9:
    df[var].fillna(9,inplace=True)
    
#for this vars: no data == 99
vars99 = ['NUM_CONSUL','APGAR1', 'APGAR2','EDAD_MADRE','NIV_EDUM','N_HIJOSV',
          'NIV_EDUP','N_HIJOSM','N_EMB']
for var in vars99:
    df[var].fillna(99,inplace=True)
    
df['ID_VIOL'].fillna('01999',inplace=True)
df['EDAD_PADRE'].fillna(999,inplace=True)
df['MANERA_MUERTE'].fillna(0,inplace=True)

df.loc[:,df.isna().any()].columns

Index(['CAUSA_667', 'CAU_HOMOL', 'C_ANT1', 'C_ANT12', 'C_ANT2', 'C_ANT22',
       'C_ANT3', 'C_ANT32', 'C_BAS1', 'C_DIR1', 'C_DIR12', 'C_MCM1', 'C_PAT1',
       'C_PAT2', 'EMB_FAL', 'EMB_MES', 'EMB_SEM', 'FECHA_NACM', 'GRU_ED1',
       'GRU_ED2', 'IDADMISALUD', 'MUERTEPORO', 'NIVEL_EDU_FALLECIDO',
       'OCUPACION', 'SIMUERTEPO', 'ULTCURFAL'],
      dtype='object')

### solo tienen sentido para nofet: GRU_ED1 y 2, NIVEL_EDU_FALLECIDO, ULTCURFAL, MUERTEPORO, SIMUERTEPO, OCUPACION, EMB_FAL, EMB_SEM, EMB_MES

## Let's clean the SIT and OTRO_SIT variables. These indicate where the event (birth, death) happened, and can be joined into a single column. Cleaning of this var needs to be done as there are some spelling mistakes or different expressions for the same place.

In [17]:
#Let's do some real cleaning

#Si alguna de las palabras en uniq_vals está contenida en "WORD", reemplaza WORD por dicha palabra
uniq_vals = ['AMBUL','CARRO','LANCHA','CARRETERA','VEREDA','TAXI','BUS','PARTERA',
             'TRABAJO','CASA','VIA','LANCHA','CARCEL','BONGO','HOTEL',
             'HOSPITAL','IGLESIA','VEHICULO','DE SALUD','INDIGENA','RESTAURANTE',
             'CLINICA','CALLE','HUERTA','BARRIO','CLINISANITAS',
             'PATRULLA','BANO','RESGUARDO','BOTE','CABILDO','DOMICILIO','V/',
             'ALMACEN','VERDA','FINCA','CANOA','PLAYA','VIVIENDA','PARQUEADERO','RURAL','RIO',
             'INTERPERIE','ALVERGUE','PUENTE','RIO','PUESTODE SALUD','CENTRODE SALUD','IPS','AUTOMOVIL',
             'PANGA','PORTE PUBLICO','SININFORMACION','POLICIA','SERVICIO PUBLICO','FLOTA','MUELLE',
             'CAMIONETA','CAMPO','COMUNIDAD','SIN INFORMACION','GERIATRICO','ANCIAN','GERONTO','HOGAR',
             'ONCOLOGICA','RESIDENCIA','FUNDACION','CAMINO','ASILO','OFICINA','AULA','BOSQUE','PARROQUIA',
             'OTRO','COMERCI','PUBLICO','BAR','LICOR','CULTIVO','DEPORTE','ANSIANATO','TABERNA','CANTINA',
             'FABRICA']

#Reemplaza explicitamente otras ocurrencias
to_replace = {'SIN INFORMACION':'NO_INFO','SIN ESTABLECER':'NO_INFO','VIA':'CALLE','AVENIDA':'CALLE',
              'AUTOPISTA':'CALLE','V/':'VEREDA','VERDA':'VEREDA','AUTOMOVIL':'VEHICULO',
              'MEDIO DE TRASPORTE ACUATICO (C':'BOTE','BARCO':'BOTE','UTOMOVIL':'VEHICULO',
              'CARRO':'VEHICULO','CANOA':'BOTE','CARETERA':'CALLE','VIVIENDA':'CASA',
              'SIN INFROMACION':'NO_INFO','AMBUL':'AMBULANCIA','SE DESCONOCE':'NO_INFO','PANGA':'BOTE',
              'CENTRO DE SLUD':'INSTITUCION DE SALUD','EMBARCACION':'BOTE','CENTRO PENITENCIARIO':'CARCEL',
              'TRANSMILENIO':'TRASPORTE PUBLICO','ALVERGUE':'ALBERGUE','CLINISANITAS':'INSTITUCION DE SALUD',
              'BUS':'TRANSPORTE PUBLICO','CHALUPA':'BOTE','SININFORMACION':'NO_INFO',
              'SERVICIO PUBLICO':'TRANSPORTE PUBLICO','FLOTA':'TRANSPORTE PUBLICO','CARRETERA':'CALLE',
              'CAMIONETA':'VEHICULO','PORTE PUBLICO':'TRANSPORTE PUBLICO','DESCONOCIDO':'NO_INFO',
              'DE SALUD':'INSTITUCION DE SALUD','LANCHA':'BOTE','INDIGENA':'COMUNIDAD INDIGENA',
              'DOMICILIO':'CASA','BONGO':'BOTE','RESIDENCIA':'CASA','HOSPITAL':'INSTITUCION DE SALUD',
              'SIN INFORMACION':'NO_INFO','GERIATRICO':'HOGAR GERIATRICO','ANCIAN':'HOGAR GERIATRICO',
              'GERONTO':'HOGAR GERIATRICO','ONCOLOGICA':'INSTITUCION DE SALUD','PARROQUIA':'IGLESIA',
              'FINCA':'RURAL','VEREDA':'RURAL','COMERCI':'COMERCIO','PUBLICO':'OTRO LUGAR PUBLICO',
              'CULTIVO':'AREA DE CULTIVO','LICOR':'BAR','DEPORTE':'AREA DEPORTE',
              'ANSIANATO':'HOGAR GERIATRICO','TABERNA':'BAR','CANTINA':'BAR'}

def clean_otrosit(df,uniq_vals,to_replace):
    import unicodedata
    def replace_isin(x):
        #Elimina caracteres especiales
        s = ''.join((c for c in unicodedata.normalize('NFD',x) if unicodedata.category(c) != 'Mn'))
        #Convierte a word de uniq_vals
        for word in uniq_vals:
            if word in s:
                return word
        return s

    #Toma solo valores para los que 'SITIO_EVENTO' indique OTRO
    a = df[df['SITIO_EVENTO']=='OTRO']['OTRO_SIT'].copy()
    a = a.apply(lambda x: replace_isin(x))
    a.replace(to_replace.keys(),to_replace.values(),inplace=True) #Reemplaza explicitamente de dict

    init_len = df['OTRO_SIT'].unique().shape[0] #Amount of unique values before replacements
    
    #Set new values
    cp_df = df.copy()
    cp_df.loc[a.index,'SITIO_EVENTO'] = a.values

    final_len = cp_df['SITIO_EVENTO'].unique().shape[0]  #unique values after
    
    #Print summary of cleaning
    print(f"Initial unique values: {init_len}\nFinal unique values: {final_len}")
    
    #Test cleaning. Check everything looks ok
    print(cp_df[cp_df['SITIO_EVENTO']!='INSTITUCION DE SALUD'][['SITIO_EVENTO','OTRO_SIT','SIT']].head(10))

    #Now, we can drop OTRO_SIT AND SIT
    cp_df.drop(columns=['OTRO_SIT','SIT'],inplace=True)
    return cp_df

df = clean_otrosit(df,uniq_vals,to_replace)

Initial unique values: 44
Final unique values: 27
     SITIO_EVENTO        OTRO_SIT SIT
29           CASA         NO_INFO   3
438          CASA         NO_INFO   3
801          CASA         NO_INFO   3
980          CASA         NO_INFO   3
1034         CASA         NO_INFO   3
1618         CASA         NO_INFO   3
1654         CASA         NO_INFO   3
2122   AMBULANCIA  AMBULANCIA TAB   6
2164         CASA         NO_INFO   3
2397        CALLE       CARRETERA   6


## It may be possible to clean some columns that contain no information after the slicing of the nofet df. (ocupacion, nivel edu fallecido, etc)

In [18]:
#First drop columns that only contain NaNs. I expect this from variables related to pregnancy when dead
print("Empty columns:")
print(df.loc[:,df.isna().all()].columns.to_list())

df = df.loc[:,~df.isna().all()]
print()
### Check other columns that are suspicios of not containing any relevant information.
#GRU_ED2 because it should always be == 1 or NaN
for col in ['EST_CIVIL','GRU_ED2','NIVEL_EDU_FALLECIDO','ULTCURFAL','MAN_MUER']:
    print(col ,df[col].unique())

Empty columns:
['EMB_FAL', 'EMB_MES', 'EMB_SEM', 'MUERTEPORO', 'SIMUERTEPO']

EST_CIVIL [9 '5' '9']
GRU_ED2 [nan '01']
NIVEL_EDU_FALLECIDO [nan '13' '99' '03']
ULTCURFAL [nan '0' '99' '6']
MAN_MUER [9 '2' '5' '3' '4' '9']


### Can only drop EST_CIVIL, GRU_ED2. Interestingly enough, there is someone in 6th grade (probably a spelling mistake). I'll drop these 2 columns as well

In [19]:
df = df.drop(columns=['EST_CIVIL','GRU_ED2','NIVEL_EDU_FALLECIDO','ULTCURFAL'])

### Lets create some dummy files in order to reuse this data cleaning later on other datasets.

In [20]:
df.to_csv("../Data/CleanData/InitialClean.csv")

In [ ]:

df.to_csv("../Data/CleanData/Test2.csv")

In [ ]:
def replaceValues(df,toNumbers = True):
    dfReplaced = df.copy()
    for field in replacements:
        print(field)
        if(toNumbers):
            dfReplaced[field] = dfReplaced[field].replace(replacements[field].values(),replacements[field].keys())            
        else:
            dfReplaced[field] = dfReplaced[field].replace(replacements[field])
    return dfReplaced

In [ ]:
dfDescriptions = replaceValues(df,toNumbers=False)

df.to_csv("../Data/CleanData/Test2_Desc.csv")